In [0]:
!rm -rf *

In [0]:
import os
import cv2
import numpy as np

!rm *.zip
!rm -R data
!mkdir data
!mkdir data/train
!mkdir data/test

# MNIST
# https://drive.google.com/uc?authuser=0&id=***************&export=download
# !wget --no-check-certificate 'https://drive.google.com/uc?id=14wt2jNlJ6O_mD4iwDuFWGUHLq26fgg6r&export=download' -O train_0.zip
# !wget --no-check-certificate 'https://drive.google.com/uc?id=1J4zzM9riershoFWP8JHduV0xltiuk51W&export=download' -O train_1.zip
# !wget --no-check-certificate 'https://drive.google.com/uc?id=1XCj4ud40si9-wlFhvjGlCR1VBNdC7YE5&export=download' -O test_0.zip
# !wget --no-check-certificate 'https://drive.google.com/uc?id=1JLLxyAgK8RwoVehTTMqH16QILOfDNK-8&export=download' -O test_1.zip

# piano
# !wget --no-check-certificate 'https://drive.google.com/uc?id=1PotHcr_gDYkS5aNL36_KKLwuVcSUEYBD&export=download' -O train_0.zip
# !wget --no-check-certificate 'https://drive.google.com/uc?id=1XKrS7whwu4INmAL-mmGRa9GN5Nn6Z54G&export=download' -O train_1.zip
# !wget --no-check-certificate 'https://drive.google.com/uc?id=1FV86TVW4z4R3i6f344D1Ksw1qLOFLu2I&export=download' -O test_0.zip
# !wget --no-check-certificate 'https://drive.google.com/uc?id=1DKSNJRsGjktJkBPxWz6lmN5sUj6X5D_I&export=download' -O test_1.zip

# jubin key0 key1
!wget --no-check-certificate 'https://drive.google.com/uc?id=1WPzC9wBToLwMe7vLmc-SLpqnQyEtW62C&export=download' -O train_1.zip
!wget --no-check-certificate 'https://drive.google.com/uc?id=1QOd3cry_1c-6Jgdyl5bcFCAG9pTSra-q&export=download' -O train_0.zip
!wget --no-check-certificate 'https://drive.google.com/uc?id=1qXcbu7SVCNVWsuMATH_pzyn3T_2Kdj7-&export=download' -O test_1.zip
!wget --no-check-certificate 'https://drive.google.com/uc?id=1Z3weW6eXHTx6ck7fey7BlJt6Nyxn8Is2&export=download' -O test_0.zip




print("Extracting training data...")
!unzip train_0.zip -d data/train>/dev/null
!unzip train_1.zip -d data/train>/dev/null

print("Extracting testing data...")
!unzip test_0.zip -d data/test>/dev/null
!unzip test_1.zip -d data/test>/dev/null
print("Extracted successfully")

!rm train_0.zip
!rm train_1.zip
!rm test_0.zip
!rm test_1.zip

print("Done")

In [0]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 20
num_classes = 2
epochs = 2

# input image dimensions
img_rows, img_cols = 300, 300
input_shape = (img_rows, img_cols, 1)

#golbal var
name_label = []
    
for label in os.listdir('data/train/'):
  for img in os.listdir('data/train/'+label):
    name_label.append(['data/train/'+label+"/"+img, label])
npArray = np.array(name_label)
# print(npArray)
print(npArray.shape)
np.random.shuffle(npArray)

npArraySize = npArray.shape[0]
# print(size)
startIndexList = range(0, int(npArraySize/batch_size)*batch_size, batch_size)
# batchGenerator
def batchGenerator():
  for i in startIndexList:
    image_arr = []
    label_arr = []
    for j in range(i,i+batch_size,1):
#         print(npArray[j][0],end=' ')
      img = cv2.imread(npArray[j][0], cv2.IMREAD_GRAYSCALE)
      image_arr.append(img)
      label_arr.append(int(npArray[j][1]))
    x_train = np.array(image_arr)
    y_train = np.array(label_arr)
#     print("before yield: batch = ", i)
    yield (x_train, y_train)

In [0]:
np.array(cv2.imread(npArray[0][0], cv2.IMREAD_GRAYSCALE)).shape

In [0]:
# x_test data extraction
image_arr_test = []
label_arr_test = []
for label in os.listdir('data/test/'):
  for img in os.listdir('data/test/'+label):
    path = 'data/test/'+label+"/"+img
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    image_arr_test.append(img)
    label_arr_test.append(int(label))
x_test = np.array(image_arr_test)
y_test = np.array(label_arr_test)

In [0]:
# Model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# For BatchGenerator that yields a smaller batch of samples at a time
for e in range(epochs):
  print("epoch %d" % e)
  # call for batchGenerator
  b = 0
  for x_train, y_train in batchGenerator():
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)
    x_train = x_train.astype('float32')
    x_train /= 255
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
#     model.fit_generator(batchGenerator(), samples_per_epoch=int(npArray.shape[0]/batch_size)*batch_size, nb_epoch=2)
    score = model.train_on_batch(x_train, y_train)
#     model.fit(x_train, y_train,
#               epochs=1,
#               verbose=0,
#               validation_split=0.05)
    model.save("my-model.h5")
    b += 1
    print(b ,score)

In [0]:
from keras.models import load_model
model = load_model("my-model.h5")
# (x_test, y_test) = (c ,d)
x_test = np.array(image_arr_test)
y_test = np.array(label_arr_test)

x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_test = x_test.astype('float32')
x_test /= 255
y_test = keras.utils.to_categorical(y_test, num_classes)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
from google.colab import files
files.download('my-model.h5')

In [0]:
!ls -lh

In [0]:
###################
# Yet to be tried #
###################
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a text file.
uploaded = drive.CreateFile({'title': 'Sample file.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))